In [2]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [4]:
trs = soup.select('#list-skin > form:nth-child(4) > table > tbody > tr')
len(trs)

100

In [5]:
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source)
trs = soup.select('.aos-init')
len(trs)

100

In [7]:
tr = trs[0]

In [8]:
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [9]:
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [10]:
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [11]:
subscriber_cnt = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber_cnt

'8730만'

In [12]:
view_cnt = tr.select_one('.view_cnt').get_text().strip()
video_cnt = tr.select_one('.video_cnt').get_text().strip()
view_cnt, video_cnt

('305억3350만', '507개')

In [13]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [14]:
convert('335억8260만')

33582600000

In [15]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber_cnt = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view_cnt = convert(tr.select_one('.view_cnt').get_text().strip())
    video_cnt = convert(tr.select_one('.video_cnt').get_text().strip())
    lines.append({
        '순위': rank, '카테고리':category, '채널명': channel, '구독자수': subscriber_cnt, '조회수': view_cnt,
        '비디오': video_cnt
    })

In [16]:
df1 = pd.DataFrame(lines)
df1.tail(7)

,순위,카테고리,채널명,구독자수,조회수,비디오
93,94,음식/요리/레시피,떵개떵,4890000,2981590000,4910
94,96,키즈/어린이,ToyMart TV,4860000,1993230000,687
95,95,키즈/어린이,뽀로로(Pororo),4860000,6853150000,4160
96,97,애완/반려동물,SBS TV동물농장x애니멀봐,4830000,4638100000,4164
97,98,키즈/어린이,CoCosToy 꼬꼬스토이,4770000,2983640000,686
98,99,음식/요리/레시피,Cooking tree 쿠킹트리,4760000,487580000,1450
99,100,미분류,매일맛나 delicious day,4740000,482360000,325


In [18]:
import time
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(2)
driver.get(url)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [ ]:
lines = []
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')
    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber_cnt = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view_cnt = convert(tr.select_one('.view_cnt').get_text().strip())
        video_cnt = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({
            '순위': rank, '카테고리':category, '채널명': channel, '구독자수': subscriber_cnt, '조회수': view_cnt,
            '비디오': video_cnt
        })
driver.close()

In [19]:
df = pd.DataFrame(lines)
df.shape

(100, 6)

In [20]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오
95,95,키즈/어린이,뽀로로(Pororo),4860000,6853150000,4160
96,97,애완/반려동물,SBS TV동물농장x애니멀봐,4830000,4638100000,4164
97,98,키즈/어린이,CoCosToy 꼬꼬스토이,4770000,2983640000,686
98,99,음식/요리/레시피,Cooking tree 쿠킹트리,4760000,487580000,1450
99,100,미분류,매일맛나 delicious day,4740000,482360000,325


In [21]:
df.to_csv('data/20230428_YouTube랭킹.csv', index=False)